# Geographic Embeddings for Poisson GLM — A Complete Workflow

**Author:** Claudio Senatore Reso  
**Inspired by:** Blier-Wong et al. (2021) — *Geographic Ratemaking with Spatial Embeddings*

---

## What This Notebook Is About

Imagine you are an actuary trying to predict how many insurance claims will occur across different geographic locations. Intuitively, where a policy is located matters enormously: a house in a flood-prone coastal area should be priced differently from one in a quiet inland suburb. The challenge is that **geography is hard to encode well in a traditional statistical model**.

This notebook walks you through a modern, practical solution to that challenge: **geographic embeddings** learned by a neural autoencoder, which are then plugged into a classic Poisson GLM for claim frequency modelling.

By the end, you will understand:
- Why raw geographic coordinates or hand-crafted spatial variables often fail to capture spatial risk
- What an autoencoder is and why it is useful for representation learning
- How to train an autoencoder on spatial features and extract compact embeddings
- How to incorporate those embeddings into a Poisson GLM
- How to evaluate whether the embeddings actually improve predictive performance

---

## The Core Problem: Geography Is Messy

Traditional GLMs work well with structured, low-dimensional inputs like age, vehicle type, or building age. But geography creates three specific difficulties:

**1. Raw coordinates are not meaningful features.**  
If you simply include latitude and longitude in a GLM, you are assuming that claim risk changes linearly as you move north or east. That is almost never true. Risk can be high in one small region and low two kilometres away, for reasons that a linear model cannot capture.

**2. Spatial features interact in complex, nonlinear ways.**  
Flood risk, population density, elevation, and income are all spatially correlated, but their *joint* effect on claims is nonlinear. Manually engineering interaction terms is tedious and prone to omitting important combinations.

**3. High dimensionality is impractical in a GLM.**  
You might have dozens of spatial covariates (census variables, hazard indices, terrain metrics). Adding all of them directly to a GLM results in overfitting, multicollinearity, and uninterpretable coefficients.

---

## The Solution: Learn a Spatial Representation

Instead of feeding raw spatial features directly into the GLM, we first **compress them** into a small, dense vector — called an **embedding** — using an autoencoder neural network.

An **autoencoder** is a neural network trained to reconstruct its own input. It does this by passing the data through a narrow bottleneck (the latent layer), which forces the network to learn a compact representation that captures the most important structure in the data. In our case:

- **Input:** 8 spatial features per location (coordinates, hazard indices, demographics, terrain)
- **Bottleneck:** 5-dimensional embedding vector
- **Output:** Reconstruction of the original 8 features

The bottleneck forces the network to summarise what is most geographically distinctive about each location in just 5 numbers. Once the autoencoder is trained, we discard the decoder and use only the encoder to transform each policy's location into a 5-dimensional embedding.

Those 5 embedding dimensions are then added as features to a Poisson GLM. The GLM remains fully interpretable and auditable, but it now benefits from a richer, more nuanced representation of spatial risk.

---

## Workflow Overview

```
Raw spatial features  ──►  Autoencoder (train to reconstruct)  ──►  Embeddings
                                                                          │
Traditional policy features  ─────────────────────────────────────────►  GLM
                                                                          │
                                                                    Claim frequency predictions
```

Let's begin.

---
## Section 1 — Imports and Setup

We need four groups of libraries:
- **NumPy and Pandas** for data manipulation
- **Matplotlib** for visualisation
- **PyTorch** for building and training the autoencoder
- **Statsmodels and Scikit-learn** for the GLM and evaluation metrics

We also fix random seeds so that results are reproducible every time the notebook is run.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error

# Fix random seeds for reproducibility.
# Any random operations in NumPy or PyTorch will produce the same results
# each time the notebook is run from scratch.
np.random.seed(42)
torch.manual_seed(42)

print("Libraries loaded successfully.")

---
## Section 2 — Simulating a Realistic Spatial Dataset

We do not have a real insurance dataset, so we will **simulate** one that has the key properties we care about:

- Policies are distributed across a spatial grid defined by longitude and latitude
- Each location has several spatial covariates (demographic, hazard, terrain)
- Each policy also has traditional non-spatial features (insured value, building age, policy count)
- Claim counts are drawn from a Poisson distribution whose rate is a **nonlinear function of both spatial and non-spatial features**

By constructing the data this way, we know the ground truth: spatial features genuinely matter for claims. A model that ignores spatial information will underperform one that captures it — which is exactly what we want to demonstrate.

### What we are simulating

| Variable | Description |
|---|---|
| `longitude`, `latitude` | Geographic coordinates (synthetic, in −100 to +100 range) |
| `pop_density` | Population density — higher near the centre of the grid |
| `median_income` | Median income — varies sinusoidally with geography |
| `flood_hazard` | A composite hazard index derived from coordinates |
| `storm_exposure` | Hazard amplified by random noise |
| `elevation` | Quadratic function of coordinates plus noise |
| `terrain_roughness` | Numerical gradient of elevation |
| `insured_value` | Policy-level insured value (non-spatial) |
| `building_age` | Age of the insured building (non-spatial) |
| `policy_count` | Number of policies at the location (non-spatial) |
| `exposure` | Risk exposure (used as offset in the Poisson model) |
| `claim_count` | Target variable — the number of claims |

In [ ]:
# Total number of policy records to simulate
N = 60_000

# ── Spatial coordinates ──────────────────────────────────────────────────────
# We use a synthetic coordinate system (−100 to +100) rather than real-world
# lat/lon. The actual scale does not matter for the methodology.
longitude = np.random.uniform(-100, 100, N)
latitude  = np.random.uniform(-100, 100, N)

# ── Spatial covariates ───────────────────────────────────────────────────────
# Population density peaks at the centre of the grid (0, 0) and decreases
# toward the edges via a Gaussian kernel. We add multiplicative noise to
# make it more realistic.
pop_density = (
    np.exp(-(longitude**2 + latitude**2) / 20_000)
    * np.random.uniform(10, 300, N)
)

# Median income varies sinusoidally — a simple way to introduce spatial
# correlation without a flat or trivially predictable pattern.
median_income = (
    35_000
    + 10_000 * np.sin(longitude / 20)
    + 8_000  * np.cos(latitude  / 25)
)

# Flood hazard: a smooth combination of sine and cosine of the coordinates,
# clipped to a [0, 3] range to avoid extreme values.
flood_hazard = np.clip(np.sin(longitude / 15) + np.cos(latitude / 15), 0, 3)

# Storm exposure adds random variation on top of flood hazard.
# This captures the idea that a high-hazard area is not uniformly exposed
# every year — there is inherent randomness in storm occurrence.
storm_exposure = flood_hazard * np.random.rand(N)

# Elevation increases quadratically away from the centre, plus Gaussian noise.
elevation = 0.005 * (longitude**2 + latitude**2) + 5 * np.random.randn(N)

# Terrain roughness approximates local variation in elevation.
# np.gradient gives the rate of change — high values indicate rugged terrain.
terrain_roughness = np.abs(np.gradient(elevation))

# ── Non-spatial policy features ───────────────────────────────────────────────
# These are the traditional actuarial rating variables that do not depend
# on location. They will be used in both the baseline and enhanced GLMs.
insured_value = np.random.uniform(50_000, 400_000, N)
building_age  = np.random.randint(0, 80, N)
policy_count  = np.random.randint(1, 4, N)

# Exposure: used as an offset in the Poisson GLM.
# In practice this might be years of coverage or number of insured units.
# We model claim_count ~ Poisson(rate * exposure), so log(exposure) becomes
# a fixed offset in the linear predictor.
exposure = np.random.randint(1, 5, N)

# ── Construct the true underlying rate ───────────────────────────────────────
# The true rate is the exponent of a sum of linear terms from both
# non-spatial and spatial features. This is a log-linear model,
# exactly what a Poisson GLM with log link assumes.
#
# A model that ignores spatial features will be missing the second half
# of this equation, which will show up as higher prediction error.

linear_traditional = (
    0.000002 * insured_value
    + 0.05   * building_age
    + 0.3    * policy_count
)

linear_spatial = (
    0.3  * pop_density
    + 0.5 * flood_hazard
    + 0.2 * storm_exposure
    + 0.1 * elevation
)

true_linear_predictor = linear_traditional + linear_spatial

# Convert the linear predictor to a rate via the inverse log link (exp)
true_rate = np.exp(true_linear_predictor)

# Draw actual claim counts from a Poisson distribution.
# The Poisson distribution naturally handles count data: non-negative integers
# with variance equal to the mean.
claim_count = np.random.poisson(true_rate * exposure)

# ── Assemble into a DataFrame ─────────────────────────────────────────────────
df = pd.DataFrame({
    "longitude":         longitude,
    "latitude":          latitude,
    "pop_density":       pop_density,
    "median_income":     median_income,
    "flood_hazard":      flood_hazard,
    "storm_exposure":    storm_exposure,
    "elevation":         elevation,
    "terrain_roughness": terrain_roughness,
    "insured_value":     insured_value,
    "building_age":      building_age,
    "policy_count":      policy_count,
    "exposure":          exposure,
    "claim_count":       claim_count
})

print(f"Dataset shape: {df.shape}")
print(f"\nClaim count distribution:")
print(df["claim_count"].describe())
df.head()

### A brief look at the spatial structure

Before building any model, it is good practice to visualise the spatial distribution of the data. Let's look at flood hazard across the grid — this is one of the dominant drivers of claims in our simulation.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot flood hazard across the spatial grid.
# A scatter plot coloured by hazard level reveals the spatial structure
# we are trying to capture.
sc = axes[0].scatter(
    df["longitude"], df["latitude"],
    c=df["flood_hazard"], cmap="YlOrRd",
    s=1, alpha=0.5
)
plt.colorbar(sc, ax=axes[0], label="Flood Hazard")
axes[0].set_title("Spatial Distribution of Flood Hazard")
axes[0].set_xlabel("Longitude")
axes[0].set_ylabel("Latitude")

# Histogram of claim counts.
# Most policies have few claims; the distribution is right-skewed,
# which is typical for insurance data and motivates the Poisson model.
axes[1].hist(df["claim_count"], bins=50, color="steelblue", edgecolor="white")
axes[1].set_title("Distribution of Claim Counts")
axes[1].set_xlabel("Claim Count")
axes[1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()

---
## Section 3 — Train / Test Split

We split the data into a **training set** (70%) used to fit the autoencoder and the GLMs, and a **test set** (30%) held out for final evaluation.

**Important:** The autoencoder will be trained only on the training portion of the spatial features. We then apply the trained encoder to both training and test sets. This is the correct approach — if we used the test set during autoencoder training, our evaluation would be optimistic.

In [ ]:
# Randomly sample 70% of rows for training
train_df = df.sample(frac=0.7, random_state=42)

# The test set is everything that was not selected for training
test_df = df.drop(train_df.index)

print(f"Training set size:  {len(train_df):,} rows ({len(train_df)/len(df)*100:.0f}%)")
print(f"Test set size:      {len(test_df):,} rows ({len(test_df)/len(df)*100:.0f}%)")

---
## Section 4 — Training the Spatial Autoencoder

### What is an autoencoder?

An autoencoder is a neural network with a specific structure:

```
Input (8 features)  →  Encoder  →  Latent vector (5 dims)  →  Decoder  →  Reconstructed input (8 features)
```

The network is trained to **reconstruct its own input**. Because the latent vector is smaller than the input, the network is forced to learn a compressed representation that captures the essential structure of the data. Reconstruction quality is measured by the mean squared error between the original input and the reconstruction.

Once training is complete:
- The **encoder** has learned to summarise each location's spatial profile in 5 numbers
- The **decoder** is discarded — we only ever use the encoder going forward

### Why 5 latent dimensions?

This is a hyperparameter choice. Five is small enough to avoid overfitting in the downstream GLM, but large enough to capture meaningful variation across 8 input features. In practice, you would tune this via cross-validation. For this notebook, 5 is a reasonable starting point.

### Normalisation: a practical note

In a production setting, you would standardise the input features (zero mean, unit variance) before training the autoencoder. This ensures that no single feature dominates the loss just because it has larger absolute values. We skip this step here to keep the focus on the methodology, but note that it is an important step in practice.

### Architecture

Our autoencoder uses a two-layer encoder and decoder:
- **Encoder:** Linear(8 → 64) → ReLU → Linear(64 → 5)
- **Decoder:** Linear(5 → 64) → ReLU → Linear(64 → 8)

ReLU (Rectified Linear Unit) is the activation function: it outputs zero for negative inputs and passes positive inputs unchanged. It introduces the nonlinearity that allows the network to learn complex patterns.

In [ ]:
# ── Define which spatial features will be fed into the autoencoder ─────────────
# We use all 8 spatial variables. These capture geography at multiple scales:
# raw location (lon/lat), demographics (density, income),
# hazard (flood, storm), and terrain (elevation, roughness).
SPATIAL_FEATURE_COLS = [
    "longitude",
    "latitude",
    "pop_density",
    "median_income",
    "flood_hazard",
    "storm_exposure",
    "elevation",
    "terrain_roughness"
]

# Number of spatial input features
INPUT_DIM = len(SPATIAL_FEATURE_COLS)

# Size of the compressed latent representation
LATENT_DIM = 5


class GeoAutoencoder(nn.Module):
    """
    A simple feedforward autoencoder for geographic feature compression.

    The encoder maps an 8-dimensional spatial feature vector to a
    5-dimensional latent (embedding) vector. The decoder attempts to
    reconstruct the original 8-dimensional input from the latent vector.

    After training, only the encoder is used to produce spatial embeddings
    for the downstream GLM.
    """

    def __init__(self, input_dim: int, latent_dim: int = 5):
        super().__init__()

        # Encoder: compress input to the latent bottleneck
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),  # expand to an intermediate representation
            nn.ReLU(),                  # nonlinear activation
            nn.Linear(64, latent_dim)   # compress to the bottleneck
        )

        # Decoder: attempt to reconstruct the original input from the bottleneck
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),  # expand from the bottleneck
            nn.ReLU(),                   # nonlinear activation
            nn.Linear(64, input_dim)    # reconstruct the original input
        )

    def forward(self, x: torch.Tensor):
        """
        Forward pass: encode the input, then decode it.

        Returns both the latent embedding (z) and the reconstruction,
        so we can compute the reconstruction loss during training.
        """
        latent_embedding = self.encoder(x)
        reconstruction   = self.decoder(latent_embedding)
        return latent_embedding, reconstruction


# Instantiate the model
autoencoder = GeoAutoencoder(input_dim=INPUT_DIM, latent_dim=LATENT_DIM)

print(autoencoder)
total_params = sum(p.numel() for p in autoencoder.parameters())
print(f"\nTotal trainable parameters: {total_params:,}")

### Preparing the training data and training loop

We extract the spatial features from the training set, convert them to PyTorch tensors, and wrap them in a `DataLoader` that will feed batches to the model during training.

**Batch training** processes a small subset of the data at each step rather than the full dataset. This is more computationally efficient and helps the optimiser generalise better.

We train for 20 epochs (full passes through the training data) using the **Adam optimiser**, which adapts the learning rate automatically. The loss function is **mean squared error** between the input and the reconstruction.

In [ ]:
# ── Prepare training data ─────────────────────────────────────────────────────
# Extract spatial features from the training set and convert to a PyTorch tensor.
# FloatTensor uses 32-bit precision, which is standard for neural network training.
spatial_train_tensor = torch.FloatTensor(train_df[SPATIAL_FEATURE_COLS].values)

# Wrap in a TensorDataset, then a DataLoader.
# The DataLoader shuffles the data each epoch and yields batches of 1024 rows.
train_loader = DataLoader(
    TensorDataset(spatial_train_tensor),
    batch_size=1024,
    shuffle=True
)

# ── Configure training ────────────────────────────────────────────────────────
# Adam is a popular gradient-based optimiser that adapts its learning rate
# based on the history of gradients. lr=1e-3 is a common default.
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

# Mean Squared Error loss: penalises the squared difference between the
# original input and the reconstruction. Lower is better.
reconstruction_loss_fn = nn.MSELoss()

# Track loss per epoch for plotting
epoch_losses = []

# ── Training loop ─────────────────────────────────────────────────────────────
print("Training the autoencoder...\n")
NUM_EPOCHS = 20

for epoch in range(NUM_EPOCHS):
    total_batch_loss = 0.0
    num_batches = 0

    for (batch_features,) in train_loader:
        # Forward pass: get the latent embedding and the reconstruction
        latent_embedding, reconstruction = autoencoder(batch_features)

        # Compute the reconstruction loss
        loss = reconstruction_loss_fn(reconstruction, batch_features)

        # Backward pass and weight update
        optimizer.zero_grad()  # clear gradients from the previous step
        loss.backward()        # compute gradients via backpropagation
        optimizer.step()       # update model weights

        total_batch_loss += loss.item()
        num_batches += 1

    # Average loss across all batches in this epoch
    mean_epoch_loss = total_batch_loss / num_batches
    epoch_losses.append(mean_epoch_loss)

    print(f"  Epoch {epoch + 1:02d}/{NUM_EPOCHS} — Reconstruction Loss: {mean_epoch_loss:.4f}")

print("\nTraining complete.")

In [ ]:
# Plot the training loss curve.
# A steadily decreasing curve confirms that the autoencoder is learning.
# If the curve flattens early, the model may have converged or may need
# more capacity (wider / deeper layers).

plt.figure(figsize=(8, 4))
plt.plot(range(1, NUM_EPOCHS + 1), epoch_losses, marker="o", color="steelblue")
plt.title("Autoencoder Training Loss (Reconstruction MSE per Epoch)")
plt.xlabel("Epoch")
plt.ylabel("Mean Reconstruction Loss")
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.show()

---
## Section 5 — Extracting the Spatial Embeddings

Now that the autoencoder is trained, we use the **encoder only** to transform the spatial features of every policy into a 5-dimensional embedding vector.

We do this for both the training and test sets. These 5 embedding dimensions become new features — `emb_0` through `emb_4` — that will be added to the Poisson GLM.

**Key point:** We wrap this step in `torch.no_grad()` because we are only performing a forward pass — we do not need to compute gradients, and skipping that saves memory and computation.

In [ ]:
# We use torch.no_grad() to disable gradient computation.
# During inference (not training), we only need the forward pass output.
with torch.no_grad():
    # Encode the training set spatial features
    train_spatial_tensor = torch.FloatTensor(train_df[SPATIAL_FEATURE_COLS].values)
    train_embeddings = autoencoder.encoder(train_spatial_tensor).numpy()

    # Encode the test set spatial features
    test_spatial_tensor = torch.FloatTensor(test_df[SPATIAL_FEATURE_COLS].values)
    test_embeddings = autoencoder.encoder(test_spatial_tensor).numpy()

# Each row is now a 5-dimensional vector
print(f"Shape of training embeddings: {train_embeddings.shape}")
print(f"Shape of test embeddings:     {test_embeddings.shape}")

# Attach the 5 embedding dimensions as new columns in the DataFrames.
# We name them emb_0 through emb_4 for clarity.
# Note: we use .copy() to avoid modifying a slice of the original DataFrame.
train_df = train_df.copy()
test_df  = test_df.copy()

for i in range(LATENT_DIM):
    train_df[f"emb_{i}"] = train_embeddings[:, i]
    test_df[f"emb_{i}"]  = test_embeddings[:, i]

print(f"\nNew embedding columns added: {[f'emb_{i}' for i in range(LATENT_DIM)]}")
train_df[[f"emb_{i}" for i in range(LATENT_DIM)]].describe()

### Visualising the embeddings

Let's do a quick check: do the learned embeddings vary coherently across space? We can plot two embedding dimensions against each other, coloured by a known spatial variable like flood hazard. If the embeddings have captured meaningful spatial structure, we should see some pattern.

In [ ]:
# Sample a subset for faster plotting
sample = train_df.sample(5000, random_state=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot emb_0 vs emb_1, coloured by flood_hazard
# If the embedding dimensions correlate with flood hazard, this plot will show
# a gradient — evidence that the autoencoder has captured spatial risk structure.
sc = axes[0].scatter(
    sample["emb_0"], sample["emb_1"],
    c=sample["flood_hazard"], cmap="YlOrRd",
    s=5, alpha=0.6
)
plt.colorbar(sc, ax=axes[0], label="Flood Hazard")
axes[0].set_title("Embedding Dimensions 0 vs 1\n(coloured by Flood Hazard)")
axes[0].set_xlabel("emb_0")
axes[0].set_ylabel("emb_1")

# Plot emb_0 vs emb_2, coloured by pop_density
sc2 = axes[1].scatter(
    sample["emb_0"], sample["emb_2"],
    c=sample["pop_density"], cmap="viridis",
    s=5, alpha=0.6
)
plt.colorbar(sc2, ax=axes[1], label="Population Density")
axes[1].set_title("Embedding Dimensions 0 vs 2\n(coloured by Population Density)")
axes[1].set_xlabel("emb_0")
axes[1].set_ylabel("emb_2")

plt.tight_layout()
plt.show()

---
## Section 6 — Fitting the Baseline Poisson GLM

### Why a Poisson GLM?

Claim counts are non-negative integers — exactly the kind of data for which a Poisson GLM is designed. The model assumes:

$$
\text{claim\_count}_i \sim \text{Poisson}(\lambda_i)
$$

where the rate $\lambda_i$ is linked to the linear predictor via the **log link**:

$$
\log(\lambda_i) = \beta_0 + \beta_1 x_{i1} + \ldots + \beta_p x_{ip} + \log(\text{exposure}_i)
$$

The $\log(\text{exposure})$ term is an **offset** — a predictor whose coefficient is fixed at 1. It adjusts for the fact that policies with more exposure (e.g., more years of coverage) naturally tend to have more claims.

### The baseline model

The baseline model uses only traditional, non-spatial features: `insured_value`, `building_age`, and `policy_count`. This represents how a traditional actuary would build the model — without any geographic information.

We will compare this against the enhanced model in Section 7.

In [ ]:
# Patsy formula syntax:
#   - Left of ~ is the response variable (claim_count)
#   - Right of ~ lists the predictors
#   - offset(np.log(exposure)) adds log-exposure as a fixed offset
#     (not estimated — its coefficient is constrained to 1)

formula_baseline = (
    "claim_count ~ "
    "insured_value + building_age + policy_count "
    "+ offset(np.log(exposure))"
)

# Fit the baseline GLM on the training set
baseline_glm = smf.glm(
    formula=formula_baseline,
    data=train_df,
    family=smf.families.Poisson()  # Poisson distribution with log link (default)
).fit()

print(baseline_glm.summary())

---
## Section 7 — Fitting the Enhanced Poisson GLM with Spatial Embeddings

The enhanced model extends the baseline by adding the 5 learned embedding dimensions (`emb_0` through `emb_4`) as additional predictors.

From the GLM's perspective, these are just five new numeric features. The GLM estimates a coefficient for each one. But unlike raw coordinates, these features encode complex, nonlinear spatial structure — the autoencoder has already done the hard work of compressing geography into a representation that is well-suited for a linear model to consume.

This is the key insight of the approach: **the representation learning happens outside the GLM** (in the autoencoder), and the GLM only needs to learn a linear mapping from the compact embedding space to claim frequency.

In [ ]:
# Build the list of embedding column names programmatically
embedding_col_names = [f"emb_{i}" for i in range(LATENT_DIM)]

# Construct the enhanced formula by appending the embedding terms
formula_enhanced = (
    "claim_count ~ "
    "insured_value + building_age + policy_count + "
    + " + ".join(embedding_col_names)
    + " + offset(np.log(exposure))"
)

print("Enhanced formula:")
print(formula_enhanced)
print()

# Fit the enhanced GLM on the training set
enhanced_glm = smf.glm(
    formula=formula_enhanced,
    data=train_df,
    family=smf.families.Poisson()
).fit()

print(enhanced_glm.summary())

---
## Section 8 — Evaluation and Comparison

We now compare the two models on the **held-out test set**.

We use two metrics:

**1. Mean Squared Error (MSE):**  
The average squared difference between the actual and predicted claim counts. Lower is better. This is easy to interpret but penalises large errors heavily.

**2. Calibration plot:**  
A scatter plot of actual vs. predicted claims. A well-calibrated model should have points clustered along the diagonal (predictions match actuals). We also show the distribution of prediction errors.

Because we know the ground truth in this simulation, we expect the enhanced model to significantly outperform the baseline — spatial features were a major driver of claims in the data generating process.

In [ ]:
# Generate predictions on the test set for both models
predictions_baseline = baseline_glm.predict(test_df)
predictions_enhanced = enhanced_glm.predict(test_df)

# Compute MSE for each model
mse_baseline = mean_squared_error(test_df["claim_count"], predictions_baseline)
mse_enhanced = mean_squared_error(test_df["claim_count"], predictions_enhanced)

print("=" * 45)
print(f"  Baseline GLM  — Test MSE: {mse_baseline:,.2f}")
print(f"  Enhanced GLM  — Test MSE: {mse_enhanced:,.2f}")
improvement_pct = 100 * (mse_baseline - mse_enhanced) / mse_baseline
print(f"  MSE Improvement:           {improvement_pct:.1f}%")
print("=" * 45)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# ── Plot 1: Calibration scatter — Baseline ───────────────────────────────────
axes[0].scatter(
    test_df["claim_count"], predictions_baseline,
    alpha=0.2, s=3, color="steelblue"
)
# A perfect model would have all points on this diagonal line
max_val = max(test_df["claim_count"].max(), predictions_baseline.max())
axes[0].plot([0, max_val], [0, max_val], "r--", label="Perfect calibration")
axes[0].set_title(f"Baseline GLM\nMSE = {mse_baseline:,.1f}")
axes[0].set_xlabel("Actual Claim Count")
axes[0].set_ylabel("Predicted Claim Count")
axes[0].legend()

# ── Plot 2: Calibration scatter — Enhanced ────────────────────────────────────
axes[1].scatter(
    test_df["claim_count"], predictions_enhanced,
    alpha=0.2, s=3, color="darkorange"
)
axes[1].plot([0, max_val], [0, max_val], "r--", label="Perfect calibration")
axes[1].set_title(f"Enhanced GLM (with Embeddings)\nMSE = {mse_enhanced:,.1f}")
axes[1].set_xlabel("Actual Claim Count")
axes[1].set_ylabel("Predicted Claim Count")
axes[1].legend()

# ── Plot 3: Distribution of prediction errors ─────────────────────────────────
errors_baseline = test_df["claim_count"].values - predictions_baseline.values
errors_enhanced = test_df["claim_count"].values - predictions_enhanced.values

axes[2].hist(errors_baseline, bins=60, alpha=0.6, color="steelblue",  label="Baseline",  density=True)
axes[2].hist(errors_enhanced, bins=60, alpha=0.6, color="darkorange", label="Enhanced",  density=True)
axes[2].axvline(0, color="red", linestyle="--", label="Zero error")
axes[2].set_title("Distribution of Prediction Errors\n(Actual − Predicted)")
axes[2].set_xlabel("Error")
axes[2].set_ylabel("Density")
axes[2].legend()

plt.tight_layout()
plt.show()

---
## Section 9 — Summary and Key Takeaways

### What we built

We implemented a two-stage modelling pipeline for insurance claim frequency:

1. **A spatial autoencoder** trained on 8 geographic features, which compresses each location into a 5-dimensional embedding. The autoencoder is trained unsupervised — it learns from the spatial features alone, without using claim counts at all.

2. **Two Poisson GLMs** — one using only traditional policy features, and one augmented with the learned spatial embeddings.

### What we demonstrated

The enhanced GLM outperformed the baseline because:
- The simulation was designed with spatial features as major claim drivers
- The autoencoder successfully compressed nonlinear spatial structure into a form the linear GLM could use
- The 5 embedding dimensions act as a learned spatial smoother — a data-driven alternative to hand-crafted spatial interaction terms

### Why this approach is practically useful

| Challenge | How geo-embeddings help |
|---|---|
| Raw coordinates are not meaningful features | Coordinates become inputs to a learned function, not direct predictors |
| Spatial features interact nonlinearly | The autoencoder captures interactions; the GLM sees only the compressed result |
| High-dimensional spatial data | Compressed to a small, fixed-size vector regardless of input dimensionality |
| Model interpretability | GLM structure is preserved; embeddings are additional numeric inputs |

### Limitations and extensions

- **No feature standardisation:** In practice, standardise inputs before training the autoencoder. This avoids features with larger absolute values dominating the loss.
- **No hyperparameter tuning:** The latent dimension (5) and architecture were chosen heuristically. In production, these should be cross-validated.
- **Unsupervised pretraining:** The autoencoder is trained without seeing claim counts. A supervised variant (e.g., an encoder trained jointly with the GLM) could in principle capture more task-relevant spatial structure.
- **Grid-level embeddings:** In a real application, you might train the autoencoder on geographic grid cells rather than individual policies, then join embeddings to policies by location. This is the approach described in Blier-Wong et al. (2021).

---

### Reference

Blier-Wong, C., Cossette, H., Lamontagne, L., & Marceau, E. (2021).  
*Geographic Ratemaking with Spatial Embeddings.*  
ASTIN Bulletin: The Journal of the IAA.